In [1]:
from ultralytics.utils import LOGGER
from ultralytics import YOLO
import cv2

import numpy as np

# yolo 로그 메시지 비활성화
LOGGER.disabled = True

In [3]:
# YOLOv8 모델 로드 (사전 학습된 COCO 데이터셋 사용)
model = YOLO('yolov8n.pt')  # 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt' 중 선택 가능

In [11]:
# 2. 로컬 영상 파일 열기
video_path = '../../../Data/video/1-1_006-C04.mp4'
url = 'rtsp://210.99.70.120:1935/live/cctv002.stream'
cap = cv2.VideoCapture(url)

In [12]:
# 3. 프레임별 처리
object_tracks = {}  # 객체 위치 저장용 딕셔너리
TARGET_CLASS = [2, 5, 7]
count_num = 0

frame_count = 0
skip_frames = 5  # 5프레임마다 한 번씩 처리

print("영상시작")

# 첫 번째 프레임에서 프레임 크기 초기화
ret, frame = cap.read()
if not ret:  # 첫 번째 프레임 읽기 실패 시 프로그램 종료
    print("비디오 파일을 읽을 수 없습니다.")
    cap.release()
    exit()

frame_height, frame_width = frame.shape[:2]
mid_x = frame_width // 2  # 화면 중간 x좌표 계산
mid_y = frame_height // 2  # 화면 중간 y좌표 계산

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    if frame_count % skip_frames != 0:  # 프레임 스킵
        continue
    if frame_count % 1000 == 0:  # 1000프레임마다 스트림 재연결
        print("스트림 초기화")
        cap.release()
        cap = cv2.VideoCapture(url)

    # YOLOv8 객체 탐지
    results = model(frame)
    detections = results[0].boxes

    # 현재 프레임에서 탐지된 객체 정보 저장
    current_frame_tracks = {}
    threshold = frame_height // 20  # 중앙선 근처 허용 오차

    for det in detections:
        x1, y1, x2, y2 = map(int, det.xyxy[0].tolist())  # 경계 상자 좌표
        confidence = det.conf[0]  # 신뢰도
        class_id = int(det.cls[0])  # 클래스id

        if confidence > 0.5 and class_id in TARGET_CLASS:
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            # object_id =  hash(f"{class_id}_{center_x}_{center_y}")  # 고유 ID 생성
            object_id = f"{class_id}_{x1}_{y1}_{x2}_{y2}"
            current_frame_tracks[object_id] = (center_x, center_y)

            #이전 위치와 비교하여 이동 방향 계산
            if object_id in object_tracks:
                prev_center_x, prev_center_y = object_tracks[object_id]
                print("object_id", object_id)
                print("object_tracks[object_id]", object_tracks[object_id])
                print("prev_center_x", prev_center_x)
                print("prev_center_y", prev_center_y)

                 # 디버깅: 객체 이동 방향 확인
                print(f"객체 {object_id}: prev_center_y={prev_center_y}, center_y={center_y}, mid_y={mid_y}")

                # 위쪽-> 아래쪽 이동
                if prev_center_y < mid_y - threshold and center_y >= mid_y + threshold:
                    count_num += 1
                    print(f"객체 {object_id}가 위쪽에서 아래쪽으로 이동. 현재 인원수: {count_num}")

                # 아래쪽 -> 위쪽 이동
                elif prev_center_y > mid_y + threshold and center_y <= mid_y - threshold:
                    count_num -= 1
                    print(f"객체 {object_id}가 아래쪽에서 위쪽으로 이동. 현재 인원수: {count_num}")

            # 현재 위치 업데이트
            # object_tracks[object_id] = (center_x, center_y)
            object_tracks.update(current_frame_tracks)

            # 경계 상자 및 중심점 표시
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
            cv2.circle(frame, (center_x, center_y), 5, (255,0, 0), -1)
    
    # # 객체 이동 방향 업데이트
    object_tracks = current_frame_tracks            
    
    
    # 영상 중간 부분에 세로선 그리기
    # cv2.line(frame, (mid_x, 0), (mid_x, frame_width), (255, 0, 0), 2)  # (255, 0, 0)은 파란색, 두께는 2
    
    # 영상 중간 부분에 가로선 그리기
    cv2.line(frame, (0, mid_y), (frame_width, mid_y), (255, 0, 0), 1)  # (255, 0, 0)은 파란색, 두께는 2

    # 프레임 보여주기
    cv2.imshow('Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


영상시작
object_id 7_262_104_314_167
object_tracks[object_id] (288, 135)
prev_center_x 288
prev_center_y 135
객체 7_262_104_314_167: prev_center_y=135, center_y=135, mid_y=240
object_id 7_262_104_314_167
object_tracks[object_id] (288, 135)
prev_center_x 288
prev_center_y 135
객체 7_262_104_314_167: prev_center_y=135, center_y=135, mid_y=240
object_id 7_262_103_320_166
object_tracks[object_id] (291, 134)
prev_center_x 291
prev_center_y 134
객체 7_262_103_320_166: prev_center_y=134, center_y=134, mid_y=240
object_id 7_262_103_320_166
object_tracks[object_id] (291, 134)
prev_center_x 291
prev_center_y 134
객체 7_262_103_320_166: prev_center_y=134, center_y=134, mid_y=240
object_id 7_262_103_320_166
object_tracks[object_id] (291, 134)
prev_center_x 291
prev_center_y 134
객체 7_262_103_320_166: prev_center_y=134, center_y=134, mid_y=240
object_id 7_262_104_314_166
object_tracks[object_id] (288, 135)
prev_center_x 288
prev_center_y 135
객체 7_262_104_314_166: prev_center_y=135, center_y=135, mid_y=240
objec